# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [10]:
import sklearn

from azureml.core import Dataset, Environment, Experiment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from pprint import pprint

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
# create a TabularDataset from a dataset

dataset = Dataset.get_by_name(ws, name='heart-failure')
heart_failure = dataset.to_pandas_dataframe()

# preview the first 3 rows of the dataset
heart_failure.head(3)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
automl_settings = {
    "task": 'classification',
    "iterations": 30,
    "experiment_timeout_minutes": 15,
    "primary_metric": 'accuracy',
    "training_data": dataset,
    "label_column_name": 'DEATH_EVENT',
    "n_cross_validations": 5,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=False)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_194d8a33-aa09-4f40-82c8-91889e31058d',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-30T10:37:37.218269Z',
 'endTimeUtc': '2021-01-30T10:53:05.461565Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-136576","workspace_name":"quick-starts-ws-136576","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":30,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":2,"fea

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [6]:
# Retrieve and save your best automl model.
best_automl_run, best_model = remote_run.get_output()
print(best_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('24',
                                           Pipeline(memory=None,
                                                    steps=[('robustscaler',
                                                            RobustScaler(copy=True,
                                                                         quantile_range=[25,
                                                                                         75],
                                                                         with_centering=True,
                                                                         with_scaling=False)),
                                                           ('extratreesclassifier',
                                                            ExtraTreesClassifier(bootstrap=False,
                                                                                 ccp_alpha=0.0,
              

In [11]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['24', '8', '9', '3', '19'],
 'weights': [0.16666666666666666,
             0.16666666666666666,
             0.3333333333333333,
             0.16666666666666666,
             0.16666666666666666]}

24 - robustscaler
{'copy': True,
 'quantile_range': [25, 75],
 'with_centering': True,
 'with_scaling': False}

24 - extratreesclassifier
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 0.8,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.01,
 'min_samples_split': 0.

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [68]:
#TODO: Save the best model
model = best_automl_run.register_model(model_path='outputs/model.pkl', model_name='capstone_automl')

print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-136576', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-136576'), name=capstone_automl, id=capstone_automl:1, version=1, tags={}, properties={})


In [69]:
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', 'environment.yml')

In [72]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

environment = Environment.from_conda_specification('my-automl-environment', './outputs/conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=environment)

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = model.deploy(ws, "automl-service", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://faea935f-66cc-4347-b6d8-74356504c30a.southcentralus.azurecontainer.io/score
http://faea935f-66cc-4347-b6d8-74356504c30a.southcentralus.azurecontainer.io/swagger.json


In [82]:
# service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [76]:
import json

#Import test data

test_df = heart_failure.sample(3) # data is the pandas dataframe of the original data
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"age": 70.0, "anaemia": 0, "creatinine_phosphokinase": 69, "diabetes": 0, "ejection_fraction": 40, "high_blood_pressure": 0, "platelets": 293000.0, "serum_creatinine": 1.7, "serum_sodium": 136, "sex": 0, "smoking": 0, "time": 75}, {"age": 60.0, "anaemia": 0, "creatinine_phosphokinase": 2261, "diabetes": 0, "ejection_fraction": 35, "high_blood_pressure": 1, "platelets": 228000.0, "serum_creatinine": 0.9, "serum_sodium": 136, "sex": 1, "smoking": 0, "time": 115}, {"age": 45.0, "anaemia": 0, "creatinine_phosphokinase": 2413, "diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 140000.0, "serum_creatinine": 1.4, "serum_sodium": 140, "sex": 1, "smoking": 1, "time": 280}]}


In [77]:
import requests

# Set the content type
headers = {'Content-type': 'application/json'}

response = requests.post(service.scoring_uri, test_sample, headers=headers)
print(response.text)

"{\"result\": [0, 0, 0]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [78]:
logs = service.get_logs()
print(logs)


2021-01-30T13:21:55,916431300+00:00 - iot-server/run 
2021-01-30T13:21:55,921541100+00:00 - gunicorn/run 
2021-01-30T13:21:55,924788600+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-30T13:21:55,939613500+00:00 - rsyslog/run 
rsyslogd

In [21]:
service.delete()